In [4]:

import json
from datetime import date, datetime, timedelta
from time import time
import pytz



limit1 = datetime.strptime("9:00:00", "%H:%M:%S").time()
limit2 = datetime.strptime("20:00:00", "%H:%M:%S").time()
limit3 = datetime.strptime("12:00:00", "%H:%M:%S").time()



doctor_time_tables = {
    "Dr. João Santos": {"monday": {'start': '09:00:00', 'end': "13:00:00"}, "wednesday": {'start': '18:00:00','end': "20:00:00"}},
    "Dr. Miguel Costa": {"monday": {'start': "13:30:00", 'end': "17:30:00"}, "thursday": {'start': '09:00:00', 'end': "13:00:00"}},
    "Dra. Sofia Pereira": {"monday": {'start': "18:00:00", 'end': "20:00:00"}, "thursday": {'start': '13:30:00', 'end': "17:30:00"}},
    "Dra. Mariana Chagas": {"tuesday": {'start': "09:00:00", 'end': "13:00:00"}, "thursday": {'start': '18:00:00', 'end': "20:00:00"}},
    "Dr. António Oliveira": {"tuesday": {'start': "13:30:00", 'end': "17:30:00"}, "friday": {'start': '09:00:00', 'end': "13:00:00"}},
    "Dra. Inês Martins": {"tuesday": {'start': "18:00:00", 'end': "20:00:00"}, "friday": {'start': '13:30:00', 'end': "17:30:00"}},
    "Dr. Ângelo Rodrigues": {"wednesday": {'start': "09:00:00", 'end': "13:00:00"}, "friday": {'start': '18:00:00', 'end': "20:00:00"}},
    "Dr. José Dias": {"wednesday": {'start': "13:30:00", 'end': "17:30:00"}, "saturday": {'start': '09:00:00', 'end': "12:00:00"}},
}


In [19]:
def is_time_in_range(selected_time, time_list):
    selected_time = datetime.strptime(selected_time, "%H:%M:%S").time()
    
    for time in time_list:
        time_obj = datetime.strptime(time, "%H:%M:%S").time()
        if time_obj == selected_time:
            return True
    return False



In [6]:
def get_available_time_slots(doctor, day):
    # Function to get available time slots for a specific doctor on a given day
    doctor_schedule = doctor_time_tables.get(doctor, {}).get(day, {})
    
    if not doctor_schedule:
        return []

    start_time = datetime.strptime(doctor_schedule.get('start', '00:00:00'), "%H:%M:%S").time()
    end_time = datetime.strptime(doctor_schedule.get('end', '23:59:59'), "%H:%M:%S").time()

    available_times = []
    current_time = start_time

    while current_time <= end_time:
        available_times.append(current_time.strftime("%H:%M:%S"))
        current_time = (datetime.combine(date.today(), current_time) + timedelta(minutes=30)).time()

    return available_times

In [7]:
def get_available_times_message(doctor, day):
    available_times = get_available_time_slots(doctor, day)
    if not available_times:
        return f"{doctor} is not available on {day}."
    return f"{doctor} is available on {day} at the following times: {', '.join(available_times)}."


In [49]:
def choose_doctor_based_on_time(requested_time_str, requested_doctor, requested_date_str):
    available_doctors = []
    requested_date = datetime.strptime(requested_date_str, "%Y-%m-%d").strftime("%A").lower()   # Monday
    requested_time = datetime.strptime(requested_time_str, "%H:%M:%S").time()  # 09:00:00

    for doctor, schedules in doctor_time_tables.items():
        for day, time_range in schedules.items():
            if day == requested_date:
                start_time = datetime.strptime(time_range['start'], "%H:%M:%S").time()
                end_time = datetime.strptime(time_range['end'], "%H:%M:%S").time()

                if start_time <= requested_time <= end_time:
                    available_doctors.append(doctor)

    if not available_doctors:
        return None  # No available doctors for the requested time

    if requested_doctor in available_doctors:
        return requested_doctor

    return None  # The originally reques

In [50]:
updated_doctor = choose_doctor_based_on_time('09:00:00', 'Dr. João Santos', '2024-01-01')
print(updated_doctor)

monday
Dr. João Santos
{'start': '09:00:00', 'end': '13:00:00'}
Dr. João Santos
